### TIFU KNN
Один из самых эффективных алгоритмов для решения задачи next basket recommendation  
Была использована его вариация  
Примерный алгоритм следующий:
* Каждую корзину кодируем как вектор, где на i-й позиции стоит количество i-го товара в этой корзине
* Далее складываем все корзины каждого пользователя:
    - Либо как взвешенное среднее по всем корзинам
    - Либо как сумму векторов, умноженных на веса
    - !Старые корзины имеют меньший коэффициент
* По полученным векторам находим ближайших соседей каждого пользователя и:
    - Либо берем итоговый вектор пользователя как взвешенное среднее по всем соседям и самому пользователю (чем дальше сосед, чем меньше коэффициент)
    - Либо находим взвешенное среднее по всем соседям, а дальше итоговый вектор получаем как P = v_user * alpha + v_avg_neighbors * (1 - alpha)
* Далее находим позиции в итоговом векторе каждого пользователя, где значения максимальны - это и есть рекомендации  
  
В оригинальном алгоритме корзины каждого пользователя разбиваются на группы, но так как в среднем покупок у пользователя не так много, этот шаг был убран

Статья о TIFU KNN (2020): https://arxiv.org/pdf/2006.00556.pdf

! Можно запускать в режиме "Run all"

In [74]:
%%time
%pylab inline

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.neighbors import NearestNeighbors

from scipy import sparse
from tqdm import tqdm
tqdm.pandas()
%pip install implicit
from implicit.lmf import LogisticMatrixFactorization
print('success!')

Populating the interactive namespace from numpy and matplotlib


C:\Users\Asya\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Note: you may need to restart the kernel to use updated packages.
success!
Wall time: 2.35 s


In [75]:
import json
filepath = 'C:/Users/Asya/Desktop/CODING/GitHub_projects/mts-teta-nbr/data/item_mapping.json'
mapping = {}    
with open(filepath, "r") as fp:
        mapping0 = json.load(fp)
demo_items = [int(i) for i in list(mapping0.keys())]
demo_items.sort()
print(demo_items)        

[0, 5, 9, 14, 15, 16, 17, 19, 21, 22, 23, 25, 26, 27, 29, 30, 31, 41, 42, 43, 54, 55, 57, 61, 82, 84, 85, 86, 88, 89, 92, 100, 169, 170, 198, 376, 379, 380, 382, 383, 384, 388, 395, 396, 398, 400, 402, 409, 411, 412, 420, 425, 430, 431, 432, 437, 440, 712, 798, 804]


In [76]:
df = pd.read_csv("data/main.csv")
df=df[df['cart'].isin(demo_items)] #LEAVE JUST ITEMS WITH PICTURES
df.rename(columns={"order_completed_at":"time"}, inplace=True) # rename "order_completed_at" column to "time"
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S") # "time" column to datetime type


In [77]:
df.sort_values(by = 'user_id')

,user_id,time,cart
2422920,0,2020-07-19 09:59:17,379
2965150,0,2020-08-24 08:55:32,383
2965151,0,2020-08-24 08:55:32,409
2965154,0,2020-08-24 08:55:32,402
2965155,0,2020-08-24 08:55:32,57
...,...,...,...
3079875,19998,2020-09-01 08:12:32,84
3079876,19998,2020-09-01 08:12:32,61
3079879,19998,2020-09-01 08:12:32,420
3106846,19998,2020-09-02 15:03:23,19


In [78]:
r = 0.75
k_nearest = 30
alpha = 0.95 
top_k = 18

In [79]:
user_id_index = df['user_id'].sort_values().unique() #list if user id's
carts_count = np.size(df['cart'].unique())
users_count = np.size(df['user_id'].unique())
print('carts_count: ', carts_count)
print('users_count: ', users_count)

carts_count:  60
users_count:  19978


In [80]:
def duplicates_to_count(t):
    return t.groupby(['user_id', 'time'])['cart'].value_counts() \
                                                  .to_frame() \
                                                  .rename(columns={"cart":"count"}) \
                                                  .reset_index()

def count_to_duplicates(t):
    g = t.copy()
    g["to_explode"] = g["count"].apply(lambda x: [i for i in range(x)])
    g = g.explode("to_explode") \
         .drop(columns=["count", "to_explode"])
    return g

def make_train_targets(t, level=1):
    user_last_time = t.groupby(["user_id"])["time"].max().to_frame().reset_index()
    user_last_time["last_buy"] = 1
    
    train = pd.merge(t, user_last_time, on=["time", "user_id"], how="left")
    train = train[train["last_buy"] != 1]
    train.drop(columns=["last_buy"], inplace=True)
    
    if level >= 2:
        return make_train_targets(train, level-1)
    
    user_last_time.drop(columns=["last_buy"], inplace=True)
    
    user_last_carts = pd.merge(user_last_time, t, on=["user_id", "time"], how="left")
    
    skeleton = make_skeleton(t)
    targets = pd.merge(skeleton, user_last_carts.drop(columns=["time"]), on=["user_id","cart"], how="left")
    targets.fillna(0, inplace=True)
    targets["count"] = targets["count"].apply(lambda x: x if x <= 1 else 1).astype(int)
    targets.rename(columns={"count":"target"}, inplace=True)
    return train, targets

def make_skeleton(t):
    return t.groupby("user_id")["cart"].unique().to_frame().reset_index().explode("cart")

def user_cart_pivot_table(t):
    g = pd.pivot_table(t, columns="cart", index=["user_id", "time"], values="count", aggfunc=np.sum, fill_value=0)
    for cart in list(set(df["cart"].unique()).difference(set(t["cart"].unique()))):
        g[cart] = 0
    return g

In [81]:
df = duplicates_to_count(df)

In [82]:
train, targets = make_train_targets(df, level=1)

In [83]:
main = user_cart_pivot_table(train)

In [84]:
main = main.reindex(sorted(main.columns), axis=1).reset_index()

In [85]:
main.head()

cart,user_id,time,0,5,9,14,15,16,17,19,...,420,425,430,431,432,437,440,712,798,804
0,0,2020-07-19 09:59:17,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,2020-08-24 08:55:32,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,2019-05-08 16:09:41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2020-01-17 14:44:23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,2020-02-06 22:46:55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
main.sort_values('time', ascending=False, inplace=True)

In [87]:
main.head()

cart,user_id,time,0,5,9,14,15,16,17,19,...,420,425,430,431,432,437,440,712,798,804
133538,10497,2020-09-03 20:03:55,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
184798,19237,2020-09-03 19:42:35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138389,11107,2020-09-03 15:19:32,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
157844,13763,2020-09-03 14:35:33,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159655,14041,2020-09-03 13:43:03,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [88]:
users_all_carts = [[] for i in range(users_count)]
main_array = main.to_numpy()
for arr in tqdm(main_array):
    user_position = int(np.where(user_id_index == arr[0])[0])
    users_all_carts[user_position].append(arr[2:])
    #arr[0] is a user id; arr[2:]
    
# Теперь users_all_carts содержит для каждого пользователя набор его корзин, закодированных как вектор

100%|██████████| 187076/187076 [00:04<00:00, 46219.40it/s]


In [89]:
# Вектор пользователя формируется как среднее взвешенное всех его корзин (более старые корзины имеют меньший вес)
def get_users_one_cart_as_avg(users, r):
    users_as_avg_cart = [[] for i in range(users_count)]
    for i in tqdm(range(len(users))):
        cart = pd.Series(np.zeros(carts_count), dtype=float)
        cur_weights = 0
        for j in range(len(users[i])):
            weight = r**(j)
            cart += pd.Series(users[i][j]) * weight
            cur_weights += weight
        if cur_weights == 0:
            cur_weights = 1
        cart /= cur_weights
        users_as_avg_cart[i] = cart.tolist()
    return users_as_avg_cart

# Вектор пользователя формируется как сумма векторов, умноженных на веса (более старые корзины имеют меньший вес)
def get_users_one_cart_as_attenuation(users, r):
    users_as_attenuation_cart = [[] for i in range(users_count)]
    for i in tqdm(range(len(users))):
        cart = pd.Series(np.zeros(carts_count), dtype=float)
        for j in range(len(users[i])):
            cart += pd.Series(users[i][j]) * r**(j)
        users_as_attenuation_cart[i] = cart.tolist()
    return users_as_attenuation_cart

In [90]:
users_as_vectors = get_users_one_cart_as_avg(users_all_carts, r) ### YOU CAN CHANGE THE FUNCTION

100%|██████████| 19978/19978 [01:17<00:00, 258.14it/s]


Ближайших соседей также можно искать несколькими способами:
* NearestNeighbors из sklearn
* Матричное разложение из implicit

In [91]:
def get_neighbors_nn(users_carts, k): #GOOD
    model = NearestNeighbors()
    model.fit(users_carts)
    neighbors = model.kneighbors(users_carts, n_neighbors=k+1, return_distance=False)
    return neighbors

def get_neighbors_mf(users_carts, k):
    model = LogisticMatrixFactorization(iterations=100)
    model.fit(sparse.csr_matrix(users_carts).transpose())
    neighbors = [ [] for i in range(users_count)]
    for i in tqdm(range(users_count)):
        nbrs = model.similar_users(i, k+1)
        for tpl in nbrs:
            neighbors[i].append(tpl[0]) 
    return neighbors

In [92]:
users_as_vectors

[[0.0,
  0.5714285714285714,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5714285714285714,
  0.0,
  0.5714285714285714,
  0.5714285714285714,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5714285714285714,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.5714285714285714,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.5714285714285714,
  0.5714285714285714,
  0.0,
  0.0,
  0.0,
  0.5714285714285714,
  0.0,
  0.0,
  0.5714285714285714,
  0.5714285714285714,
  0.5714285714285714,
  0.0,
  0.0,
  0.0,
  0.42857142857142855,
  0.0,
  0.5714285714285714,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.3657142857142857,
  0.0,
  0.0,
  0.0,
  0.08790165324289953,
  0.0,
  0.0,
  0.08790165324289953,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.04944467994913099,
  0.0,
  0.0,
  0.04944467994913099,
  0.20510385756676558,
  0.0,
  0.0,
  0.0,
  0.39501483679525223,
  0.27781263247138616,
  0

In [93]:
#size(users_as_vectors)

In [94]:
neighbors = get_neighbors_nn(users_as_vectors, k_nearest) ### YOU CAN CHANGE THE FUNCTION

In [95]:
# Итоговый вектор пользователя высчитывается по формуле P = v_user * alpha + v_avg_neighbors * (1 - alpha)
def combine_with_neighbors_avg(users_carts, neighbors, alpha):
    avg_nbrs = [ [] for i in range(users_count) ]
    for i in tqdm(range(users_count)):
        avg = pd.Series(np.zeros(carts_count))
        cur_weights = 0
        for j in range(1, len(neighbors[i])):
            avg += pd.Series(users_carts[neighbors[i][j]]) * (len(neighbors[i]) - j)
            cur_weights += (len(neighbors[i]) - j)
        avg /= (cur_weights if cur_weights != 0 else 1)
        avg_nbrs[i] = avg
    return [pd.Series(users_carts[i]) * alpha + pd.Series(avg_nbrs[i]) * (1-alpha) for i in range(users_count)]

# Итоговый вектор пользователя высчитывается как сумма вектора пользователя и векторов его соседей,
# умноженных на коэффициенты (чем дальше сосед, тем меньше коэффициент)
# Этот вариант позволяет получить большее значение метрики
def combine_with_neighbors_attenuation(users_carts, neighbors, alpha):
    users_p = [ [] for i in range(users_count)]
    for i in tqdm(range(len(neighbors))):
        users_p[i] = pd.Series(users_carts[i])
        for j in range(1, k_nearest+1):
            users_p[i] += alpha**(j) * pd.Series(users_carts[neighbors[i][j]])
    return users_p

In [96]:
users_as_final_vectors = combine_with_neighbors_attenuation(users_as_vectors, neighbors, alpha) ### YOU CAN CHANGE THE FUNCTION

100%|██████████| 19978/19978 [03:35<00:00, 92.77it/s] 


In [97]:
users_best_basket = []
for i in tqdm(range(users_count)):
    users_best_basket.append(users_as_final_vectors[i].sort_values(ascending=False).head(top_k).index.tolist())

100%|██████████| 19978/19978 [00:05<00:00, 3988.09it/s]


In [98]:
users_best_basket[:1]

[[22, 3, 25, 36, 47, 24, 9, 38, 23, 39, 54, 17, 1, 52, 46, 50, 44, 43]]

Можно также добавлять к рекомендациям предметы, найденные через закономерности, см. `2. Apriori.ipynb`

In [99]:
add_carts_from_apriori = False

if add_carts_from_apriori:
    ap = pd.read_csv("data/apriori_top_20.csv")
    ap_map = {}
    for index, row in ap.iterrows():
        ap_map[int(row["from"])] = int(row["to"])
    
    for i in tqdm(range(users_count)):
        for key in ap_map.keys():
            if key in users_best_basket[i] and ap_map[key] not in users_best_basket[i]:
                users_best_basket[i].append(ap_map[key])

In [112]:
recs = pd.DataFrame()
recs["user_id"] = [user_id_index[i] for i in range(users_count)]
recs["items"] = users_best_basket

In [113]:
recs.head()

,user_id,items
0,0,"[22, 3, 25, 36, 47, 24, 9, 38, 23, 39, 54, 17,..."
1,1,"[21, 58, 32, 3, 33, 28, 51, 59, 22, 52, 34, 10..."
2,2,"[23, 22, 10, 38, 52, 4, 39, 3, 25, 44, 53, 6, ..."
3,3,"[22, 23, 44, 52, 25, 17, 5, 39, 38, 7, 3, 19, ..."
4,4,"[22, 44, 23, 25, 9, 50, 41, 3, 6, 5, 57, 52, 5..."


In [114]:
recs = recs.explode('items').rename(columns={'items': 'cart'})
recs['predict'] = 1

In [115]:
res = pd.merge(targets, recs, on=["user_id", 'cart'], how="left")

In [116]:
res['predict'] = res['predict'].fillna(0).astype(int)

In [117]:
res.head()

,user_id,cart,target,predict
0,0,14,0,0
1,0,57,1,0
2,0,82,0,0
3,0,379,0,0
4,0,430,0,0


In [118]:
round(f1_score(res["target"], res["predict"]), 5)

0.19934

#### Мини исследование алгоритмов
При фиксированных параметрах 
* r = 0.8
* k_nearest = 20              
* alpha = 0.85    
* top_k = 15
* level = 1

Менялись варианты (1) усреднения корзин пользователя, (2) поиска ближайших соседей и (3) объединения пользователя с соседями, получил следующие значения метрики
* (1)-(2)-(3) - score
* avg-nn-avg - 0.37851
* avg-nn-att - 0.39407 - best
* avg-mf-avg - 0.37861
* avg-mf-att - 0.38423
* att-nn-avg - 0.37727
* att-nn-att - 0.38949 
* att-mf-avg - 0.37817
* att-mf-att - 0.38835  

** att - attenuation - затухание, при таком подходе мы берем коэффициент затухания от новых покупок к старым 
  
Что можно сказать точно - для усреднения корзин пользователя лучше использовать get_users_one_cart_as_attenuation()

In [119]:
# r = 0.75
# k_nearest = 30
# alpha = 0.95     Scores: 0.40996, 0.41104, 0.37132 - mean=0.39744
# top_k = 18
# avg-nn-att

In [120]:
# r = 0.8
# k_nearest = 20             
# alpha = 0.85     Scores: 0.39407, 0.38983, 0.35291 - mean=0.37894
# top_k = 15
# avg-nn-att

In [121]:
# r = 0.85
# k_nearest = 26    
# alpha = 0.91     Scores: 0.39005, 0.38579, 0.34770 - mean=0.37451
# top_k = 14
# avg-nn-att

In [122]:
# r = 0.9
# k_nearest = 30    
# alpha = 0.75    Scores: 0.35904, 0.35214, 0.31353 - mean=0.34157
# top_k = 10
# avg-nn-att